In [61]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [62]:
#We import the libraries
import yfinance as yf # type: ignore
import pandas as pd # type: ignore
import numpy as np
from numpy import linalg as LA
from numpy.linalg import inv

Question 1:

In [63]:
Tickers = ['^RUT','^IXIC','^GSPC','XWD.TO']
startinput = '2014-01-01'
endinput = '2015-01-01'

S=pd.DataFrame()

for t in Tickers:
    S[t]=yf.Ticker(t).history(start=startinput, end=endinput)['Close']

# We remove the NaN values for each column
S = S.fillna(method='ffill').fillna(method='bfill')

# Show the first 5 and last 5 rows of the dataframe
display(S.head())
display(S.tail())


C:\Users\gamer\AppData\Local\Temp\ipykernel_8244\170970120.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  S = S.fillna(method='ffill').fillna(method='bfill')


,^RUT,^IXIC,^GSPC,XWD.TO
Date,,,,
2014-01-02 00:00:00-05:00,1150.719971,4143.069824,1831.979980,27.058308
2014-01-03 00:00:00-05:00,1156.089966,4131.910156,1831.369995,27.100622
2014-01-06 00:00:00-05:00,1147.160034,4113.680176,1826.770020,27.100622
2014-01-07 00:00:00-05:00,1157.630005,4153.180176,1837.880005,27.566133
2014-01-08 00:00:00-05:00,1157.459961,4165.609863,1837.489990,27.557663


,^RUT,^IXIC,^GSPC,XWD.TO
Date,,,,
2014-12-24 00:00:00-05:00,1206.780029,4773.470215,2081.879883,31.666769
2014-12-26 00:00:00-05:00,1215.209961,4806.859863,2088.770020,31.666769
2014-12-29 00:00:00-05:00,1219.109985,4806.910156,2090.570068,31.679703
2014-12-30 00:00:00-05:00,1213.050049,4777.439941,2080.350098,31.438272
2014-12-31 00:00:00-05:00,1204.699951,4736.049805,2058.899902,31.300304


In [64]:
#We compute the return vector R
R=S/S.shift(1)-1
R=R[1:]

Mean_d=R.mean()
Covar_d=R.cov()
Corr=R.corr()

In [73]:
Mean=Mean_d*252 #Annualized mean
Covar=Covar_d*252 #Annualized covariance

r=2/100 #We take r0=2%
d=len(Tickers)
Vec1 = np.linspace(1,1,d)
Sigma = Covar
InvSigma = inv(Sigma) 

#Important variables calculations

a=Vec1.T.dot(InvSigma).dot(Vec1) 
b=Vec1.T.dot(InvSigma).dot(Mean)

#Tanget portfolio
pi_T= (InvSigma.dot(Mean-r*Vec1))/(b-a*r) #Tangent Portfolio

print("The allocation of the Tangent Portfolio is:",pi_T) 

The allocation of the Tangent Portfolio is: [-1.80878998  2.07669512 -0.20745803  0.93955288]


Question 2:
1. Annualized volatilities and expected returns of the Tangent Portfolio


In [ ]:
sd_T=np.sqrt(pi_T.dot(Sigma).dot(pi_T)) #Annualized volatility of the Tangent Portfolio
mean_T=pi_T.dot(Mean)   #Annualized expected return of the Tangent Portfolio

print("Annualized volatilies of the Tangent Portfolio:", sd_T)
print("Annualized expected returns of the Tangent Portfolio:", mean_T)

Annualized volatilies of the Tangent Portfolio: 0.16939013552718102
Annualized expected returns of the Tangent Portfolio: 0.3103789977335803


2.Annualized volatilities and expected returns of the 4 assets

In [71]:
print("Expected returns of the 4 assets:\n", Mean)
print("\nVariance Covariance matrix:\n", Covar)

STD=np.sqrt(np.diag(Covar))
STD_named = pd.Series(STD, index=Tickers)
print("\nVolatilities for the 4 assets:\n", STD_named)

Expected returns of the 4 assets:
 ^RUT      0.059325
^IXIC     0.144324
^GSPC     0.123698
XWD.TO    0.152872
dtype: float64

Variance Covariance matrix:
             ^RUT     ^IXIC     ^GSPC    XWD.TO
^RUT    0.026661  0.020909  0.015474  0.012665
^IXIC   0.020909  0.020011  0.014946  0.012398
^GSPC   0.015474  0.014946  0.012893  0.010507
XWD.TO  0.012665  0.012398  0.010507  0.013272

Volatilities for the 4 assets:
 ^RUT      0.163283
^IXIC     0.141460
^GSPC     0.113547
XWD.TO    0.115204
dtype: float64


Question 3:


In [72]:
print("Sharpe ratio of the Tangent Portfolio:", (mean_T-r)/sd_T)
print("Sharpe ratio of the 4 assets:\n", (Mean-r)/STD)


Sharpe ratio of the Tangent Portfolio: 1.7142615585603855
Sharpe ratio of the 4 assets:
 ^RUT      0.240839
^IXIC     0.878861
^GSPC     0.913259
XWD.TO    1.153357
dtype: float64
